In [ ]:
cmd <- function(bash_command) {
    output <- system(bash_command, intern=TRUE)
    last_line = ""
    
    for (line in output) { 
        cat(line)
        cat("\n")
        last_line = line 
    }
    
    return(last_line) 
}

In [ ]:
cmd("pip install 'sagemaker[local]' --upgrade")

In [ ]:
s3.bucket <- "sagemaker-cookbook-bucket"
prefix <- "chapter11"

training.s3_input_location <- paste0("s3://", s3.bucket, "/", prefix ,"/input/train.r.json")
training.s3_output_location <- paste0("s3://", s3.bucket, "/", prefix ,"/output/")

In [ ]:
# optionally you can do this
training.s3_input_location <- readRDS("tmp/s3.path")
training.s3_input_location

In [ ]:
library('reticulate')

sagemaker <- import('sagemaker')

In [ ]:
container <- "581320662326.dkr.ecr.us-east-1.amazonaws.com/r-forecast:1"

In [ ]:
role <- sagemaker$get_execution_role()

In [ ]:
LocalSession <- sagemaker$local$LocalSession

session <- LocalSession()
session$config <- list(local=list(local_code=TRUE))

In [ ]:
TrainingInput <- sagemaker$inputs$TrainingInput

sagemaker.train_input <- TrainingInput(training.s3_input_location, content_type="application/json")

In [ ]:
Estimator <- sagemaker$estimator$Estimator

estimator <- Estimator(
    container,
    role, 
    instance_count=1L, 
    instance_type="local",
    output_path=training.s3_output_location,
    sagemaker_session=session)

In [ ]:
# estimator$set_hyperparameters(a=1L, b=2L, c=3L)

In [ ]:
estimator$fit(list(train = sagemaker.train_input))

In [ ]:
estimator$model_data

In [ ]:
predictor <- estimator$deploy(
    initial_instance_count=1L,
    instance_type="local",
    endpoint_name="custom-local-r-forecast-endpoint"
)

In [ ]:
response <- predictor$predict("100")
response

In [ ]:
library('rjson')

In [ ]:
endpoint.values <- fromJSON(toString(response))
endpoint.values

In [ ]:
library(forecast)

tmp.x <- ts(endpoint.values$x.values, 
            start=endpoint.values$x.start, 
            frequency=endpoint.values$x.frequency)

tmp.mean <- ts(endpoint.values$mean.values, 
               start=endpoint.values$mean.start, 
               frequency=endpoint.values$mean.frequency)

tmp=list(x=tmp.x, mean=tmp.mean)

tmp.forecast <- structure(tmp, class="forecast")
tmp.forecast

In [ ]:
options(repr.plot.width=15, repr.plot.height=6)
tmp.forecast %>% autoplot()

In [ ]:
predictor$delete_endpoint()